In [ ]:
# import required libraries

from azure.ml.entities import Workspace, CommandJob, Code, ResourceConfiguration
from azure.ml import MLClient
from azure.identity import DefaultAzureCredential

In [ ]:
# Enter details of your AML workspace

subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
# get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

## Run a command job on Singularity compute

In [ ]:
singularity_config = {
    "interactive": False,
    "userAlias": "<USER_ALIAS>",
    "scalePolicy": {
        "autoScale": False,
        "autoScaleIntervalInSec": 120,
        "maxInstanceTypeCount": 1,
        "minInstanceTypeCount": 1
    },
    "priority": "medium",
    "slaTier": "Premium",
    "enableAzmlInt": True
}

job = CommandJob(
    code=Code(local_path="./src"), # Local training scripts
    command="python train_mnist.py",
    compute="/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.MachineLearningServices/virtualclusters/<VIRTUAL_CLUSTER>",
    environment="singularity-mlflow:2", # pre-registered environment with custom docker image
    environment_variables={
        "var1":"var1",
        "USER_ALIAS_MNT_SRC":"test"
    },
    resources=ResourceConfiguration(
        instance_type="D2",
        properties={
            "Singularity": singularity_config
        })) 

# Submit a command job that will log metrics and register a trained model at the end
returned_job = ml_client.jobs.create_or_update(job)
returned_job.services["Studio"].endpoint